# Prototype an automatic bot that constantly scans and evaluates Fairlay odds.

## Setup

In [1]:
import os
if 'src' not in os.listdir():
    os.chdir("..")

In [78]:
import requests

import pandas as pd
import numpy as np

import src.fairlay
import src.load
import src.models.gp

## Fetch premium matches from Datdota.

In [85]:
matches = src.load.fetch_matches()

In [71]:
matches.df

,startDate,league_name,radiant_name,dire_name,radiantVictory,radiant_nicknames,dire_nicknames,seriesId,duration,radiant_valveId,radiant_players,dire_valveId,dire_players,league_id,startTimestamp,series_start_time,match_i_in_series
matchId,,,,,,,,,,,,,,,,,
3810442630,2018-04-01 04:00:22,Dota 2 Asia Championships 2018,TNC Predator,Vici Gaming,True,"[Raven, TIMS, Armel, Kuku, Sam H]","[old LaNm, Ori, Fenrir, old eLeVeN, Paparazi]",-3810442630,1941,2108395,"[132309493, 155494381, 164532005, 184950344, 1...",726228,"[89423756, 107803494, 113800818, 134276083, 13...",9643,1522555222000,1522555222000,0
3810503944,2018-04-01 04:59:14,Dota 2 Asia Championships 2018,PSG.LGD,OG,False,"[xNova, Chalice, fy, Somnus丶M, Ame]","[n0tail, JerAx, s4, 7ckngMad, Fly]",-3810503944,2940,15,"[94296097, 94738847, 101695162, 106863163, 177...",2586976,"[19672354, 26771994, 41231571, 88271237, 94155...",9643,1522558754000,1522558754000,0
3810509929,2018-04-01 05:04:38,Dota 2 Asia Championships 2018,INVICTUS GAMING,VGJ Thunder,True,"[Xxs, Agressif, Q, Srf, BoBoKa]","[Sylar, ddc, Freeze, Yang, Fade]",-3810509929,2385,5,"[129958758, 130416036, 140153524, 156662698, 2...",5027210,"[108382060, 114239371, 137272985, 139937922, 1...",9643,1522559078000,1522559078000,0
3810591728,2018-04-01 06:13:31,Dota 2 Asia Championships 2018,OpTic Gaming,KEEN GAMING,False,"[zai, 33, ppd, Pajkatt, CCnC]","[old chicken, yoona, 天命, MS, dark]",-3810591728,2811,5026801,"[73562326, 86698277, 86727555, 87285329, 22166...",2626685,"[135878232, 142974425, 158847773, 254710104, 3...",9643,1522563211000,1522563211000,0
3810598478,2018-04-01 06:18:32,Dota 2 Asia Championships 2018,Effect,paiN Gaming,True,"[Afoninje, Resolut1on, AfterLife, Maden, KingR]","[Kingrd, hFn, Arms, Duster, tavo]",-3810598478,1464,5197722,"[81852496, 86725175, 86785083, 93473848, 18299...",67,"[84853828, 94004717, 101062921, 116525052, 117...",9643,1522563512000,1522563512000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4985891168,2019-08-25 06:12:23,The International 2019,PSG.LGD,Team Liquid,False,"[xNova, Chalice, fy, Somnus丶M, Ame]","[MinD_ContRoL, KuroKy, w33, GH, Miracle]",360881,2194,15,"[94296097, 94738847, 101695162, 106863163, 177...",2163,"[34505203, 82262664, 86700461, 101356886, 1052...",10749,1566713543000,1566703068000,2
4986133311,2019-08-25 08:33:12,The International 2019,Team Liquid,OG,True,"[MinD_ContRoL, KuroKy, w33, GH, Miracle]","[n0tail, JerAx, 7ckngMad, topson, ana]",360940,2914,2163,"[34505203, 82262664, 86700461, 101356886, 1052...",2586976,"[19672354, 26771994, 88271237, 94054712, 31136...",10749,1566721992000,1566721992000,0
4986260666,2019-08-25 09:51:46,The International 2019,Team Liquid,OG,False,"[MinD_ContRoL, KuroKy, w33, GH, Miracle]","[n0tail, JerAx, 7ckngMad, topson, ana]",360940,1945,2163,"[34505203, 82262664, 86700461, 101356886, 1052...",2586976,"[19672354, 26771994, 88271237, 94054712, 31136...",10749,1566726706000,1566721992000,1


## Test fetching the Fairlay odds.

In [20]:
fairlay = src.fairlay.fetch_markets()
fairlay_df = src.fairlay.fairlay_json_to_df(fairlay)

In [33]:
sel = ((fairlay_df.dota_market_type == 'map') & (fairlay_df.Status == 0))
fairlay_df = fairlay_df.loc[sel].reset_index().set_index(
    ['ID', 'Title', 'wager_type', 'norm_runner'])
fairlay_df

Comp  \
ID           Title                          wager_type norm_runner                                            
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming     Dota 2 - WePlay! Reshuffle Madness   
                                            against    Nemiga Gaming     Dota 2 - WePlay! Reshuffle Madness   
180609048486 RED CODE vs. United            on         RED CODE          Dota 2 - WePlay! Reshuffle Madness   
                                            against    RED CODE          Dota 2 - WePlay! Reshuffle Madness   
180616644636 TEAM TEAM vs. Team Plus        on         TEAM TEAM      Dota 2 - ESL One Hamburg NA Qualifier   
                                            against    TEAM TEAM      Dota 2 - ESL One Hamburg NA Qualifier   
180616682679 TEAM TEAM vs. Team Plus        on         TEAM TEAM      Dota 2 - ESL One Hamburg NA Qualifier   
                                            against    TEAM TEAM      Dota 2 - ESL One Hamburg NA Qualifier   
180616682779 TEAM TEAM vs. Team Plus        on         TEAM TEAM      Dota 2 - ESL One Hamburg NA Qualifier   
                                            against    TEAM TEAM      Dota 2 - ESL One Hamburg NA Qualifier   
180616682980 Black Sheep vs. Elite 4        on         Black Sheep    Dota 2 - ESL One Hamburg NA Qualifier   
                                            against    Black Sheep    Dota 2 - ESL One Hamburg NA Qualifier   
180616683073 Black Sheep vs. Elite 4        on         Black Sheep    Dota 2 - ESL One Hamburg NA Qualifier   
                                            against    Black Sheep    Dota 2 - ESL One Hamburg NA Qualifier   
180616683170 Black Sheep vs. Elite 4        on         Black Sheep    Dota 2 - ESL One Hamburg NA Qualifier   
                                            against    Black Sheep    Dota 2 - ESL One Hamburg NA Qualifier   

                                                                        Descr  \
ID           Title                          wager_type norm_runner              
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming  1st Map   
                                            against    Nemiga Gaming  1st Map   
180609048486 RED CODE vs. United            on         RED CODE       1st Map   
                                            against    RED CODE       1st Map   
180616644636 TEAM TEAM vs. Team Plus        on         TEAM TEAM      1st Map   
                                            against    TEAM TEAM      1st Map   
180616682679 TEAM TEAM vs. Team Plus        on         TEAM TEAM      2nd Map   
                                            against    TEAM TEAM      2nd Map   
180616682779 TEAM TEAM vs. Team Plus        on         TEAM TEAM      3rd Map   
                                            against    TEAM TEAM      3rd Map   
180616682980 Black Sheep vs. Elite 4        on         Black Sheep    1st Map   
                                            against    Black Sheep    1st Map   
180616683073 Black Sheep vs. Elite 4        on         Black Sheep    2nd Map   
                                            against    Black Sheep    2nd Map   
180616683170 Black Sheep vs. Elite 4        on         Black Sheep    3rd Map   
                                            against    Black Sheep    3rd Map   

                                                                     dota_market_type  \
ID           Title                          wager_type norm_runner                      
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming              map   
                                            against    Nemiga Gaming              map   
180609048486 RED CODE vs. United            on         RED CODE                   map   
                                            against    RED CODE                   map   
180616644636 TEAM TEAM vs. Team Plus        on         TEAM TEAM                  map   
                      

## Try to match team names.

Strategy:

1. Find the most recent match with a given team name.
2. Get the player IDs of that match.

In [47]:
colnames = ['team_name', 'team_id', 'player_names', 'player_ids', 'startDate',
            'timestamp']
radi_teams = matches.df[['radiant_name', 'radiant_valveId',
                         'radiant_nicknames', 'radiant_players', 'startDate',
                         'startTimestamp']]
radi_teams.columns = colnames
dire_teams = matches.df[['dire_name', 'dire_valveId', 'dire_nicknames',
                         'dire_players', 'startDate', 'startTimestamp']]
dire_teams.columns = colnames
teams_by_date = (pd.concat([radi_teams, dire_teams]).sort_values('startDate')
                 .reset_index().drop_duplicates('team_name', keep='last'))
for colname in ('matchId', 'team_id'):
    teams_by_date[colname] = teams_by_date[colname].astype(pd.Int64Dtype())
teams_by_date.index = teams_by_date['team_name'].str.lower()
teams_by_date

,matchId,team_name,team_id,player_names,player_ids,startDate,timestamp
team_name,,,,,,,
effect,3810598478,Effect,5197722,"[Afoninje, Resolut1on, AfterLife, Maden, KingR]","[81852496, 86725175, 86785083, 93473848, 18299...",2018-04-01 06:18:32,1522563512000
hashtag.esports,3822153999,Hashtag.ESports,5241980,"[Tigger, `reNиiw, Mcnas, , บุปผาสะบั้]","[99573176, 105338676, 109469423, 115399224, 12...",2018-04-08 06:03:53,1523167433000
next e-sports,3822412089,NeXT E-sports,4993511,"[Kuroneko, Joker, ! O, _bluff`X, Unknown]","[89335663, 100781036, 115106525, 156068689, 38...",2018-04-08 09:10:47,1523178647000
sponsor.neolution,3822690978,Sponsor.Neolution,5158586,"[PIGGYSEK, Evolut1on`, PILA, Robert xD, potnoy-]","[104121929, 134112654, 135584110, 210697245, 4...",2018-04-08 12:07:45,1523189265000
improved,3823008429,Improved,5138601,"[Varen, NoctisAK47, PysiAz, Toei, Akeculsiff]","[100560596, 101239422, 127204374, 168089239, 3...",2018-04-08 14:54:17,1523199257000
...,...,...,...,...,...,...,...
vici gaming,4983550247,Vici Gaming,726228,"[Ori, Paparazi, Yang, Dy, Fade]","[107803494, 137193239, 139937922, 143693439, 1...",2019-08-24 03:08:50,1566616130000
team secret,4984038549,Team Secret,1838315,"[zai, Puppey, YapzOr, MidOne, Nisha]","[73562326, 87278757, 89117038, 116585378, 1217...",2019-08-24 09:17:18,1566638238000
psg.lgd,4985891168,PSG.LGD,15,"[xNova, Chalice, fy, Somnus丶M, Ame]","[94296097, 94738847, 101695162, 106863163, 177...",2019-08-25 06:12:23,1566713543000


## Fit the match data and predict outcomes.

In [9]:
gp_model = src.models.gp.SkillsGPMAP.from_match_df(
    matches,
    cov_func_name='exponential',
    cov_func_kwargs={'scale': 1.25 * 365 * 24 * 60 * 60 * 1000},
    radi_prior_sd=3.0,
    logistic_scale=3.0
)

> /Users/yl3/github/dota2/src/models/gp.py(349)__init__()
-> super().__init__(*args, **kwargs)
(Pdb) c


In [11]:
gp_model.fit()

In [12]:
gp_model.fitted

     fun: -129099.6211309299
     jac: array([ 1.79338611e-05, -2.34616547e-05, -7.05020867e-06, ...,
       -1.60314728e-06,  1.62650759e-06, -1.09824683e-06])
 message: 'Optimization terminated successfully.'
    nfev: 17
    nhev: 14469
     nit: 16
    njev: 32
  status: 0
 success: True
       x: array([0.14179308, 0.14182361, 0.12668286, ..., 0.046098  , 0.04558213,
       0.49055129])

### Predict outcomes.

In [48]:
matched_teams_1 = teams_by_date.reindex(fairlay_df['team_1'].str.lower())
matched_teams_2 = teams_by_date.reindex(fairlay_df['team_2'].str.lower())
display(matched_teams_1.head())
display(matched_teams_2.head())

,matchId,team_name,team_id,player_names,player_ids,startDate,timestamp
team_1,,,,,,,
nemiga gaming,4892061752,Nemiga Gaming,5014799,"[mellojul, Fervian, Zitraks, Astral, TheChosen...","[40805086, 112464030, 133558180, 197547532, 29...",2019-07-08 16:27:01,1.562603e+12
nemiga gaming,4892061752,Nemiga Gaming,5014799,"[mellojul, Fervian, Zitraks, Astral, TheChosen...","[40805086, 112464030, 133558180, 197547532, 29...",2019-07-08 16:27:01,1.562603e+12
red code,NaN,NaN,NaN,NaN,NaN,NaT,NaN
red code,NaN,NaN,NaN,NaN,NaN,NaT,NaN
team team,4668946745,TEAM TEAM,6904594,"[Newsham, Brax, EternaLEnVy, ixmike88, Gunnar]","[1296625, 31818853, 43276219, 86715129, 126238...",2019-04-19 14:18:43,1.555684e+12


,matchId,team_name,team_id,player_names,player_ids,startDate,timestamp
team_2,,,,,,,
old but gold,4611988149,Old but Gold,6711721,"[VANSKOR, Illidan, velheor, G, 633]","[11550182, 26316691, 82305765, 87586992, 96196...",2019-04-04 20:35:20,1.554410e+12
old but gold,4611988149,Old but Gold,6711721,"[VANSKOR, Illidan, velheor, G, 633]","[11550182, 26316691, 82305765, 87586992, 96196...",2019-04-04 20:35:20,1.554410e+12
united,NaN,NaN,NaN,NaN,NaN,NaT,NaN
united,NaN,NaN,NaN,NaN,NaN,NaT,NaN
team plus,NaN,NaN,NaN,NaN,NaN,NaT,NaN


In [52]:
input_df = pd.DataFrame({
    'team_1': matched_teams_1['player_ids'].values,
    'team_2': matched_teams_2['player_ids'].values,
    'startDate': matched_teams_1['startDate'].values,
    'timestamp': matched_teams_1['timestamp'].values,
    }).set_index(fairlay_df.index).dropna()
input_df

team_1  \
ID           Title                          wager_type norm_runner                                                        
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming  [40805086, 112464030, 133558180, 197547532, 29...   
                                            against    Nemiga Gaming  [40805086, 112464030, 133558180, 197547532, 29...   

                                                                                                                 team_2  \
ID           Title                          wager_type norm_runner                                                        
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming  [11550182, 26316691, 82305765, 87586992, 96196...   
                                            against    Nemiga Gaming  [11550182, 26316691, 82305765, 87586992, 96196...   

                                                                               startDate  \
ID           Title                          wager_type norm_runner                         
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming 2019-07-08 16:27:01   
                                            against    Nemiga Gaming 2019-07-08 16:27:01   

                                                                         timestamp  
ID           Title                          wager_type norm_runner                  
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming  1.562603e+12  
                                            against    Nemiga Gaming  1.562603e+12

In [53]:
fairlay_pred = gp_model.predict_matches(input_df.team_1, input_df.team_2,
                                        input_df.timestamp)

(              pred_win_prob  win_prob-2sd  win_prob+2sd  \
 timestamp                                                 
 1.562603e+12       0.590107      0.583515      0.596666   
 1.562603e+12       0.590107      0.583515      0.596666   
 
               pred_win_prob_unknown_side  radi_skill  radi_skill_sd  \
 timestamp                                                             
 1.562603e+12                    0.549724   -0.537128       0.004469   
 1.562603e+12                    0.549724   -0.537128       0.004469   
 
               dire_skill  dire_skill_sd  radi_adv  
 timestamp                                          
 1.562603e+12   -1.139796       0.040533  0.490551  
 1.562603e+12   -1.139796       0.040533  0.490551  ,
               11550182   26316691   40805086   82305765   87586992   \
 timestamp                                                             
 1.562603e+12  -0.245441  -0.289274   0.125014  -0.335817  -0.078413   
 1.562603e+12  -0.245441  -0.289274   0

## Compute additional values.

### Number of games per player in each match

In [59]:
matches_by_player = (matches.players_mat != 0).sum()
matches_by_player

1296625      110
3916428       40
3940262      148
4281729       51
5150808       61
            ... 
882896652     10
893135754      7
907097333      1
907351651      3
909485668      2
Length: 948, dtype: int64

In [97]:
input_df.team_1.apply(lambda x: matches_by_player[x].tolist())

ID            Title                           wager_type  norm_runner  
180524814883  Nemiga Gaming vs. Old But Gold  on          Nemiga Gaming    [7, 7, 15, 15, 15]
                                              against     Nemiga Gaming    [7, 7, 15, 15, 15]
Name: team_1, dtype: object

In [65]:
input_df.team_2.apply(lambda x: matches_by_player[x])

11550182  \
ID           Title                          wager_type norm_runner               
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming       149   
                                            against    Nemiga Gaming       149   

                                                                      26316691  \
ID           Title                          wager_type norm_runner               
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming       110   
                                            against    Nemiga Gaming       110   

                                                                      82305765  \
ID           Title                          wager_type norm_runner               
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming        95   
                                            against    Nemiga Gaming        95   

                                                                      87586992  \
ID           Title                          wager_type norm_runner               
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming       127   
                                            against    Nemiga Gaming       127   

                                                                      96196828  
ID           Title                          wager_type norm_runner              
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming       136  
                                            against    Nemiga Gaming       136

### The last played game of each player

In [90]:
input_df.team_1.apply(lambda x: matches.players.last_game.reindex(x))

40805086   \
ID           Title                          wager_type norm_runner                         
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming 2019-07-08 16:27:01   
                                            against    Nemiga Gaming 2019-07-08 16:27:01   

                                                                               112464030  \
ID           Title                          wager_type norm_runner                         
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming 2019-07-08 16:27:01   
                                            against    Nemiga Gaming 2019-07-08 16:27:01   

                                                                               133558180  \
ID           Title                          wager_type norm_runner                         
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming 2019-07-08 16:27:01   
                                            against    Nemiga Gaming 2019-07-08 16:27:01   

                                                                               197547532  \
ID           Title                          wager_type norm_runner                         
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming 2019-07-08 16:27:01   
                                            against    Nemiga Gaming 2019-07-08 16:27:01   

                                                                               296226696  
ID           Title                          wager_type norm_runner                        
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming 2019-07-08 16:27:01  
                                            against    Nemiga Gaming 2019-07-08 16:27:01

In [91]:
input_df.team_2.apply(lambda x: matches.players.last_game.reindex(x))

11550182  \
ID           Title                          wager_type norm_runner                         
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming 2019-07-08 14:40:22   
                                            against    Nemiga Gaming 2019-07-08 14:40:22   

                                                                                26316691  \
ID           Title                          wager_type norm_runner                         
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming 2019-07-08 14:40:22   
                                            against    Nemiga Gaming 2019-07-08 14:40:22   

                                                                                82305765  \
ID           Title                          wager_type norm_runner                         
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming 2019-07-08 14:40:22   
                                            against    Nemiga Gaming 2019-07-08 14:40:22   

                                                                                87586992  \
ID           Title                          wager_type norm_runner                         
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming 2019-07-08 14:40:22   
                                            against    Nemiga Gaming 2019-07-08 14:40:22   

                                                                                96196828  
ID           Title                          wager_type norm_runner                        
180524814883 Nemiga Gaming vs. Old But Gold on         Nemiga Gaming 2019-07-08 14:40:22  
                                            against    Nemiga Gaming 2019-07-08 14:40:22

In [96]:
import dill
with open('temp_gp_model', 'wb') as fh:
    dill.dump(gp_model, fh)